### Summary of Project, NLP_with_Wikipedia

The objective of this assignment is to engineer a novel wikipedia search engine using what you've learned about data collection, infrastructure, and natural language processing.

The project has two components:
- Data collection
- Search algorithm development

### Summary of Notebook:

The following is notebook 2 of 3 for this project, NLP_with_Wikipedia.

Steps contained in this notebook specifically:
1. Reconnect to instantiated Mongo Database
2. Clean the `text` column of each page
3. Pickle the cleaned data set for easy access in notebook 3

----------------------

### Connect to Wikipedia Collection inside Mongo DB

In [11]:
# !conda install --yes --quiet pymongo altair psycopg2
# !pip install spacy nltk
# !python -m spacy download en



Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    altair: 1.2.1-py_0   conda-forge
    vega:   0.4.4-py36_1 conda-forge

Installing /opt/conda/lib/python3.6/site-packages/vega/static -> jupyter-vega
Making directory: /opt/conda/share/jupyter/nbextensions/jupyter-vega/
Copying: /opt/conda/lib/python3.6/site-packages/vega/static/vega.js -> /opt/conda/share/jupyter/nbextensions/jupyter-vega/vega.js
Copying: /opt/conda/lib/python3.6/site-packages/vega/static/vega-lite.html -> /opt/conda/share/jupyter/nbextensions/jupyter-vega/vega-lite.html
Copying: /opt/conda/lib/python3.6/site-packages/vega/static/index.js -> /opt/conda/share/jupyter/nbextensions/jupyter-vega/index.js
Copying: /opt/conda/lib/python3.6/site-packages/vega/static/index.js.map -> /opt/conda/share/jupyter/nbextensions/jupyter-vega/index.js.map
Copying: /opt/conda/lib/python3.6/site-packages/vega/static/vega.html -> /opt/conda/share/jupyter/nbextensions/jupyter-vega/

    100% |████████████████████████████████| 37.4MB 31.0MB/s ta 0:00:0110% |▏                               | 174kB 758kB/s eta 0:00:49    7% |██▎                             | 2.6MB 9.9MB/s eta 0:00:04    10% |███▍                            | 3.9MB 5.8MB/s eta 0:00:06    13% |████▍                           | 5.2MB 16.7MB/s eta 0:00:02    24% |███████▉                        | 9.2MB 16.2MB/s eta 0:00:02    35% |███████████▎                    | 13.1MB 13.4MB/s eta 0:00:02    42% |█████████████▊                  | 16.1MB 16.7MB/s eta 0:00:02    54% |█████████████████▌              | 20.4MB 28.6MB/s eta 0:00:01    56% |██████████████████              | 21.0MB 6.7MB/s eta 0:00:03    60% |███████████████████▍            | 22.6MB 20.5MB/s eta 0:00:01    70% |██████████████████████▋         | 26.4MB 15.2MB/s eta 0:00:01    86% |███████████████████████████▊    | 32.4MB 23.9MB/s eta 0:00:01    92% |█████████████████████████████▌  | 34.5MB 16.0MB/s eta 0:00:01    93% |█████████████████████████

### Initialization

In [26]:
% run __init__.py
% run mongo.py
% run nlp.py
% matplotlib inline

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
client = pymongo.MongoClient(aws_pubIP, 27016)

In [19]:
db_ref = client.wikipedia_text
collection_ref = db_ref.my_collection

In [20]:
client.database_names(), db_ref.collection_names()

(['admin', 'local', 'test', 'wikipedia_text'], ['my_collection'])

Do not need the cursor, but instantiated just in case

In [21]:
cursor = collection_ref.find()

### Pull data from MongoDB Wikipedia Collection

Stored data from Mongo as df

In [22]:
wiki_df = pd.DataFrame(list(collection_ref.find({})))

In [29]:
nltk_stop = stopwords.words('english')

### Clean text column of `wiki_df`

In [35]:
def cleaner(text):
## Credit: Michael Frantz
    nlp = spacy.load('en')
    
    text = BeautifulSoup(text, "html5lib").get_text()
    text = re.sub('&#39;',' ',text).lower()
    text = re.sub('<br />',' ',text)
    text = re.sub('<.*>.*</.*>',' ', text)
    text = re.sub('[\d]',' ',text)
    text = re.sub('[0-9]',' ',text)
    text = re.sub('[^a-z ]',' ',text)
    text = ' '.join(i.lemma_ for i in nlp(text)
                    if i.orth_ not in nltk_stop)
    text = re.sub('\\ufeff', ' ', text)
    text = re.sub('[^a-z ]',' ',text)
    text = ' '.join(text.split())
    
    return text

In [37]:
wiki_df['text_clean'] = wiki_df['text'].map(lambda x:cleaner(x))

### Pickle cleaned `wiki_df`

In [44]:
cd ..

/home/jovyan/Documents/GA_DSI/Projects/project_4


In [45]:
wiki_df.to_pickle('data/wiki_df.p')